In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler # LabelEncoder se usará selectivamente si es necesario
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras import backend as K


In [ ]:
#leer el dataset
df = pd.read_csv('Mental Health Dataset.csv')
df.head()

,Timestamp,Gender,Country,Occupation,self_employed,family_history,treatment,Days_Indoors,Growing_Stress,Changes_Habits,Mental_Health_History,Mood_Swings,Coping_Struggles,Work_Interest,Social_Weakness,mental_health_interview,care_options
0,8/27/2014 11:29,Female,United States,Corporate,NaN,No,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Not sure
1,8/27/2014 11:31,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,No
2,8/27/2014 11:32,Female,United States,Corporate,NaN,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes
3,8/27/2014 11:37,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,Maybe,Yes
4,8/27/2014 11:43,Female,United States,Corporate,No,Yes,Yes,1-14 days,Yes,No,Yes,Medium,No,No,Yes,No,Yes


In [ ]:
!pip install ydata_profiling

In [ ]:
from ydata_profiling import ProfileReport

In [ ]:
ProfileReport(df, title="Profiling Report")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 17/17 [00:20<00:00,  1.19s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#eliminar filas duplicadas
df.drop_duplicates(inplace=True)

In [ ]:
#eliminar filas con nulos
df.dropna(inplace=True)

In [ ]:
#comprobar que no hay valores nulos
df.isnull().sum()

,0
Timestamp,0
Gender,0
Country,0
Occupation,0
self_employed,0
family_history,0
treatment,0
Days_Indoors,0
Growing_Stress,0
Changes_Habits,0


In [ ]:
#devolver el tamaño del dataset
df.shape

(284858, 17)

In [ ]:
#Vemos información genral de los datos
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 284858 entries, 3 to 292363
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Timestamp                284858 non-null  object
 1   Gender                   284858 non-null  object
 2   Country                  284858 non-null  object
 3   Occupation               284858 non-null  object
 4   self_employed            284858 non-null  object
 5   family_history           284858 non-null  object
 6   treatment                284858 non-null  object
 7   Days_Indoors             284858 non-null  object
 8   Growing_Stress           284858 non-null  object
 9   Changes_Habits           284858 non-null  object
 10  Mental_Health_History    284858 non-null  object
 11  Mood_Swings              284858 non-null  object
 12  Coping_Struggles         284858 non-null  object
 13  Work_Interest            284858 non-null  object
 14  Social_Weakness          

In [ ]:
#Eliminación de la columna Timestap ya que no aporta valor al análisis
df.drop(columns=['Timestamp'], inplace=True)

In [ ]:
#Reordenación de indices que pueden haber sufrido cambios
df.reset_index(drop=True, inplace=True)

Codificación de Variables.
Para VAE es necesario que los datos sean numéricos

La variable Country tiene muchos valores y si la codificamos con One-Hot va a dar muchas columnas así que vamos a sustituirla por una variable binaria que guarde si las personas son de estados unidos (1) u otro lugar (0). Ya que mas o menos la mitad de los datos son de EE.UU

In [ ]:
df['Country_USA'] = df['Country'].apply(lambda x: 1 if x == 'United States' else 0)
df.drop(columns=['Country'], inplace=True)#eliminación de la original

In [ ]:
df.columns

Index(['Gender', 'Occupation', 'self_employed', 'family_history', 'treatment',
       'Days_Indoors', 'Growing_Stress', 'Changes_Habits',
       'Mental_Health_History', 'Mood_Swings', 'Coping_Struggles',
       'Work_Interest', 'Social_Weakness', 'mental_health_interview',
       'care_options', 'Country_USA'],
      dtype='object')

In [ ]:
#Devolver los valores que toma la columna 'Days_Indoors'
df['Days_Indoors'].unique()

array(['1-14 days', 'Go out Every day', 'More than 2 months',
       '15-30 days', '31-60 days'], dtype=object)

In [ ]:
# 'Days_Indoors' tiene un orden implícito así que la codificaremos manualmente
days_indoors_mapping = {
    'Go out Every day': 0,
    '1-14 days': 1,
    '15-30 days': 2,
    '31-60 days': 3,
    'More than 2 months': 4
}
df['Days_Indoors'] = df['Days_Indoors'].map(days_indoors_mapping)

In [ ]:
df.dtypes

,0
Gender,object
Occupation,object
self_employed,object
family_history,object
treatment,object
Days_Indoors,int64
Growing_Stress,object
Changes_Habits,object
Mental_Health_History,object
Mood_Swings,object


In [ ]:
#El resto de varibles se codifican con One-hot
colums_codif = df.select_dtypes(include=['object']).columns
df = pd.get_dummies(df, columns=colums_codif,  dtype=int)

In [ ]:
#Verificar que no haya nulos
df.isnull().sum()

,0
Days_Indoors,0
Country_USA,0
Gender_Female,0
Gender_Male,0
Occupation_Business,0
Occupation_Corporate,0
Occupation_Housewife,0
Occupation_Others,0
Occupation_Student,0
self_employed_No,0


In [ ]:
df.dtypes

,0
Days_Indoors,int64
Country_USA,int64
Gender_Female,int64
Gender_Male,int64
Occupation_Business,int64
Occupation_Corporate,int64
Occupation_Housewife,int64
Occupation_Others,int64
Occupation_Student,int64
self_employed_No,int64


In [ ]:
df['Days_Indoors'].unique()

array([1, 0, 4, 2, 3])

In [ ]:
#Todas las columnas ahora son numericas pero necesitamos que esten entre 0 y 1 y con el mapeo Days_Indoors es la unica que no lo cumple
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)


Construcción del VAE

In [ ]:
input_dim = df.shape[1]
latent_dim = 2
# Dado que One-Hot puede aumentar mucho las dimensiones, podríamos necesitar capas más grandes o más capas.
# Para este ejemplo, mantendremos una arquitectura relativamente simple.
intermediate_dim_encoder = max(64, input_dim // 4) # Ajustar dinámicamente o fijar
intermediate_dim_decoder = max(64, input_dim // 4)
beta_kl_loss = 1.0

# --- Encoder ---
inputs = Input(shape=(input_dim,), name='encoder_input')
h_e = layers.Dense(intermediate_dim_encoder, activation='relu')(inputs)
# h_e = layers.Dense(intermediate_dim_encoder // 2, activation='relu')(h_e)
z_mean = layers.Dense(latent_dim, name='z_mean')(h_e)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(h_e)

def sampling(args):
    z_mean_layer, z_log_var_layer = args
    batch = tf.shape(z_mean_layer)[0]
    dim = tf.shape(z_mean_layer)[1]
    epsilon = K.random_normal(shape=(batch, dim), mean=0., stddev=1.0)
    return z_mean_layer + K.exp(0.5 * z_log_var_layer) * epsilon

z = layers.Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
encoder_model = Model(inputs, [z_mean, z_log_var, z], name='encoder')
print("\n--- Resumen del Encoder ---")
encoder_model.summary()

# --- Decoder ---
decoder_input_layer = Input(shape=(latent_dim,), name='decoder_input')
h_d = layers.Dense(intermediate_dim_decoder, activation='relu')(decoder_input_layer)
# h_d = layers.Dense(intermediate_dim_decoder * 2, activation='relu')(h_d)
outputs = layers.Dense(input_dim, activation='sigmoid')(h_d)
decoder_model = Model(decoder_input_layer, outputs, name='decoder')
print("\n--- Resumen del Decoder ---")
decoder_model.summary()

# --- Clase VAE ---
class VAE(Model):
    def __init__(self, encoder, decoder, beta=1.0, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.beta = beta
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [self.total_loss_tracker, self.reconstruction_loss_tracker, self.kl_loss_tracker]

    def train_step(self, data):
        x = data[0] if isinstance(data, tuple) else data
        with tf.GradientTape() as tape:
            z_mean_val, z_log_var_val, z_val = self.encoder(x)
            reconstruction = self.decoder(z_val)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(tf.keras.losses.binary_crossentropy(x, reconstruction), axis=-1)
            )
            kl_loss = -0.5 * tf.reduce_mean(
                tf.reduce_sum(1 + z_log_var_val - tf.square(z_mean_val) - tf.exp(z_log_var_val), axis=-1)
            )
            total_loss = reconstruction_loss + self.beta * kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {m.name: m.result() for m in self.metrics}

    def call(self, inputs):
        _, _, z = self.encoder(inputs)
        return self.decoder(z)


--- Resumen del Encoder ---


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, 41)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      2,688 │ encoder_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_mean (Dense)      │ (None, 2)         │        130 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_log_var (Dense)   │ (None, 2)         │        130 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z (Lambda)          │ (None, 2)         │          0 │ z_mean[0][0],     │
│                     │                   │            │ z_log_var[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,948 (11.52 KB)

 Trainable params: 2,948 (11.52 KB)

...

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ decoder_input (InputLayer)      │ (None, 2)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 41)             │         2,665 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,857 (11.16 KB)

 Trainable params: 2,857 (11.16 KB)

 Non-trainable params: 0 (0.00 B)

Visualización del Espacio Latente

In [ ]:
encoder_visualization_model = Model(inputs, z_mean)
z_points_original = encoder_visualization_model.predict(df, verbose=0)

if 'treatment' in df_scaled.columns:
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(z_points_original[:, 0], z_points_original[:, 1],
                          c=df_scaled["treatment"], cmap='coolwarm', edgecolor='k', alpha=0.6, s=10)
    plt.colorbar(scatter, label='Treatment (Escalado)')
    plt.xlabel("Dimensión Latente 1 (z_mean)")
    plt.ylabel("Dimensión Latente 2 (z_mean)")
    plt.title("Espacio Latente de Datos Originales (Coloreado por 'treatment')")
    plt.grid(True)
    plt.show()

Entrenamiento del VAE

In [ ]:
vae = VAE(encoder_model, decoder_model, beta=beta_kl_loss)
vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))

print("\nEntrenando el VAE...")
history = vae.fit(df, df, epochs=100, batch_size=64, verbose=1)


Entrenando el VAE...
Epoch 1/100
4451/4451 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - kl_loss: 1.5060 - reconstruction_loss: 34.7361 - total_loss: 36.2421
Epoch 2/100
4451/4451 ━━━━━━━━━━━━━━━━━━━━ 23s 3ms/step - kl_loss: 2.6467 - reconstruction_loss: 30.6561 - total_loss: 33.3028
Epoch 3/100
4451/4451 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - kl_loss: 2.9784 - reconstruction_loss: 29.6834 - total_loss: 32.6618
Epoch 4/100
4451/4451 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - kl_loss: 3.2978 - reconstruction_loss: 28.7848 - total_loss: 32.0826
Epoch 5/100
4451/4451 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - kl_loss: 3.5718 - reconstruction_loss: 28.0076 - total_loss: 31.5795
Epoch 6/100
4451/4451 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - kl_loss: 3.7375 - reconstruction_loss: 27.4748 - total_loss: 31.2124
Epoch 7/100
4451/4451 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - kl_loss: 3.8441 - reconstruction_loss: 27.1303 - total_loss: 30.9744
Epoch 8/100
4451/4451 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - kl_loss: 3.9370 - reconstructio

 Generación y Evaluación de Datos Sintéticos

In [ ]:
# --- 7.1. Generación de Muestras Sintéticas ---
num_synthetic_samples = len(df)
random_latent_vectors = np.random.normal(size=(num_synthetic_samples, latent_dim))
synthetic_data_scaled = decoder_model.predict(random_latent_vectors, verbose=0)

In [ ]:
# --- 7.2. Des-escalado de Datos Sintéticos ---
synthetic_data_original_scale = scaler.inverse_transform(synthetic_data_scaled)
df_synthetic_original_scale = pd.DataFrame(synthetic_data_original_scale, columns=df.columns)

In [ ]:
# --- 7.3. Post-procesamiento de Datos Sintéticos ---
# df_synthetic_original_scale ya contiene los datos des-escalados
# (resultado de scaler.inverse_transform(synthetic_data_scaled) de la celda 7.2)

df_synthetic_postprocessed = df_synthetic_original_scale.copy()

# 1. Post-procesar 'Days_Indoors' (columna mapeada manualmente)
# Los valores generados son continuos. Redondeamos al entero más cercano
# y luego limitamos al rango original del mapeo (0-4).
if 'Days_Indoors' in df_synthetic_postprocessed.columns:
    min_val_di = min(days_indoors_mapping.values()) # Debería ser 0
    max_val_di = max(days_indoors_mapping.values()) # Debería ser 4

    df_synthetic_postprocessed['Days_Indoors'] = np.round(df_synthetic_postprocessed['Days_Indoors'])
    df_synthetic_postprocessed['Days_Indoors'] = df_synthetic_postprocessed['Days_Indoors'].clip(min_val_di, max_val_di)
    print("Columna 'Days_Indoors' en datos sintéticos post-procesada (redondeada y limitada).")

# 2. Post-procesar Columnas One-Hot Encoded
# Necesitamos los nombres de las columnas originales que fueron convertidas a one-hot.
# Esto se guardó en 'colums_codif' durante tu preprocesamiento.
# Si 'colums_codif' no está disponible aquí, necesitas redefinirla o conocer esos nombres.

# Asumimos que 'colums_codif' está disponible y contiene los nombres de las columnas
# originales que se sometieron a One-Hot Encoding.
# Ejemplo: colums_codif = ['Gender', 'Occupation', 'self_employed', ...]

# Es crucial que 'colums_codif' esté definido correctamente con las columnas que realmente
# pasaron por pd.get_dummies.
# Si no tienes 'colums_codif' en este punto, puedes inferirlo buscando columnas
# en df_procesado (el dataframe justo ANTES de escalar) que tengan prefijos comunes
# y terminen con las categorías.

# Por ejemplo, si df_original tenía una columna 'Gender' con categorías 'Male', 'Female'
# y pd.get_dummies la convirtió en 'Gender_Male' y 'Gender_Female'.
# colums_codif_originales = df_original.select_dtypes(include=['object']).columns
# # Excluimos las que mapeamos manualmente o tratamos de forma especial antes de get_dummies
# columnas_que_fueron_one_hot = [
#     col for col in colums_codif_originales
#     if col not in ['Timestamp', 'Country', 'Days_Indoors', 'Mood_Swings', 'treatment'] # Ajusta esta lista según tu preprocesamiento
# ]
# print(f"Columnas originales que se asumirán como One-Hot Encoded: {columnas_que_fueron_one_hot}")


# Esta es una forma más robusta si 'colums_codif' no está disponible:
# Identificar los prefijos de las columnas one-hot a partir de df_procesado.columns
# Esto asume que las columnas one-hot tienen el formato 'PrefijoOriginal_Categoria'
original_categorical_cols_for_onehot_postprocessing = []
if 'colums_codif' in locals() or 'colums_codif' in globals():
    print(f"Usando 'colums_codif' definido previamente para el post-procesamiento One-Hot: {colums_codif.tolist()}")
    original_categorical_cols_for_onehot_postprocessing = colums_codif.tolist()
else:
    print("Advertencia: 'colums_codif' no está definido. Intentando inferir columnas One-Hot.")
    # Lógica de inferencia (puede necesitar ajustes)
    possible_prefixes = set()
    for col in df_procesado.columns: # df_procesado ANTES de escalar
        if '_' in col:
            possible_prefixes.add(col.split('_')[0])
    for prefix in possible_prefixes:
        # Verifica si el prefijo corresponde a una columna original que era categórica
        if prefix in df_original.columns and df_original[prefix].dtype == 'object' and \
           prefix not in ['Timestamp', 'Country', 'Days_Indoors', 'Mood_Swings', 'treatment']:
            original_categorical_cols_for_onehot_postprocessing.append(prefix)
    print(f"Columnas inferidas para post-procesamiento One-Hot: {original_categorical_cols_for_onehot_postprocessing}")


for prefix in original_categorical_cols_for_onehot_postprocessing:
    # Encuentra todas las columnas one-hot generadas para este prefijo original
    cols_one_hot_group = [col for col in df_synthetic_postprocessed.columns if col.startswith(prefix + '_')]

    if cols_one_hot_group:
        # Para cada fila, encontrar el índice de la columna con el valor máximo dentro del grupo
        max_indices = df_synthetic_postprocessed[cols_one_hot_group].values.argmax(axis=1)

        # Crear un array de ceros con la misma forma
        processed_one_hot_group = np.zeros_like(df_synthetic_postprocessed[cols_one_hot_group].values)

        # Poner 1 en la columna con el valor máximo para cada fila
        for i, idx in enumerate(max_indices):
            processed_one_hot_group[i, idx] = 1

        # Reemplazar las columnas originales generadas (probabilidades) con las binarias (0/1)
        df_synthetic_postprocessed[cols_one_hot_group] = processed_one_hot_group
        print(f"Columnas One-Hot para '{prefix}' post-procesadas en datos sintéticos (argmax aplicado).")

# 3. Post-procesar 'Country_IsUSA' (columna binaria creada manualmente)
# Los valores generados serán continuos entre 0 y 1. Podemos redondearlos a 0 o 1.
if 'Country_IsUSA' in df_synthetic_postprocessed.columns:
    df_synthetic_postprocessed['Country_IsUSA'] = np.round(df_synthetic_postprocessed['Country_IsUSA']).clip(0, 1)
    print("Columna 'Country_IsUSA' en datos sintéticos post-procesada (redondeada y limitada).")


print("\n--- Primeras 5 filas de datos sintéticos (después de des-escalar y post-procesamiento completo) ---")
print(df_synthetic_postprocessed.head())


Columna 'Days_Indoors' en datos sintéticos post-procesada (redondeada y limitada).
Usando 'colums_codif' definido previamente para el post-procesamiento One-Hot: ['Gender', 'Occupation', 'self_employed', 'family_history', 'treatment', 'Growing_Stress', 'Changes_Habits', 'Mental_Health_History', 'Mood_Swings', 'Coping_Struggles', 'Work_Interest', 'Social_Weakness', 'mental_health_interview', 'care_options']
Columnas One-Hot para 'Gender' post-procesadas en datos sintéticos (argmax aplicado).
Columnas One-Hot para 'Occupation' post-procesadas en datos sintéticos (argmax aplicado).
Columnas One-Hot para 'self_employed' post-procesadas en datos sintéticos (argmax aplicado).
Columnas One-Hot para 'family_history' post-procesadas en datos sintéticos (argmax aplicado).
Columnas One-Hot para 'treatment' post-procesadas en datos sintéticos (argmax aplicado).
Columnas One-Hot para 'Growing_Stress' post-procesadas en datos sintéticos (argmax aplicado).
Columnas One-Hot para 'Changes_Habits' post-

In [ ]:
# --- 7.4. Comparación Visual de Distribuciones ---
# Comparamos df_procesado (datos reales antes de escalar) con df_synthetic_postprocessed
# Asegúrate de que las columnas que comparas existen en ambos DataFrames y tienen nombres consistentes.

cols_for_comparison_post = ['Days_Indoors', 'Country_IsUSA', 'treatment'] # Columnas mapeadas/binarizadas manualmente

# Para las One-Hot, puedes comparar la distribución de una de las columnas dummy resultantes.
# Por ejemplo, si tenías 'Gender' y se convirtió en 'Gender_Female' y 'Gender_Male':
if 'Gender_Female' in df_procesado.columns and 'Gender_Female' in df_synthetic_postprocessed.columns:
    cols_for_comparison_post.append('Gender_Female')
if 'Occupation_Corporate' in df_procesado.columns and 'Occupation_Corporate' in df_synthetic_postprocessed.columns: # Ejemplo
    cols_for_comparison_post.append('Occupation_Corporate')


for col_name in cols_for_comparison_post:
    if col_name in df_procesado.columns and col_name in df_synthetic_postprocessed.columns:
        # Para una comparación justa de distribuciones, usa df_procesado (antes de escalar)
        # y df_synthetic_postprocessed (después de des-escalar y post-procesar)
        plot_distribution_comparison(df_procesado[col_name].astype(float),
                                     df_synthetic_postprocessed[col_name].astype(float),
                                     col_name)
    else:
        print(f"Advertencia: La columna '{col_name}' no se pudo comparar (no encontrada en ambos DataFrames).")

NameError: name 'df_procesado' is not defined